<a href="https://colab.research.google.com/github/laurapabon03/Final-IA-/blob/main/Copy_of_Final_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Parcial Final Inteligencia Artificial

Ángela Sofia Torres, Juan David Saldaña Rivera, Laura Pabón, Dario Montoya, Lucas Alvarado

In [1]:
# SECCIÓN 1: INSTALACIÓN DE DEPENDENCIAS
# Estas líneas instalan las bibliotecas necesarias para el funcionamiento del sistema
# Se ejecutan solamente en entornos como Google Colab o Jupyter Notebooks

# Instalación de bibliotecas esenciales para RAG
!pip install -q langchain langchain_community sentence-transformers pypdf python-docx docx2txt unstructured faiss-cpu gradio
!pip install -q chromadb requests

In [2]:
# SECCIÓN 2: CONFIGURACIÓN DE OLLAMA
# Ollama es una herramienta que permite ejecutar modelos de lenguaje localmente con menor uso de recursos que otros sistemas

# Instalación de Ollama
print("Instalando Ollama para inferencia rápida...")
!curl -fsSL https://ollama.com/install.sh | sh

# Iniciar el servicio de Ollama en segundo plano
print("\nIniciando servidor Ollama...")
!pkill ollama || true  # Detener cualquier instancia anterior
!nohup /usr/local/bin/ollama serve > ollama_output.log 2>&1 &  # Iniciar en segundo plano

# Dar tiempo al servidor para inicializar
import time
print("Esperando a que el servidor Ollama esté listo...")
time.sleep(15)  # Esperar 15 segundos

# Verificar que el servidor esté activo
print("\nVerificando que el servidor Ollama esté respondiendo...")
!curl -s http://localhost:11434/api/tags || echo "El servidor Ollama no está respondiendo"

# Descargar el modelo LLM que usaremos (Llama2)
print("\nDescargando modelo llama2 desde Ollama...")
!ollama pull llama2

Instalando Ollama para inferencia rápida...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

Iniciando servidor Ollama...
Esperando a que el servidor Ollama esté listo...

Verificando que el servidor Ollama esté respondiendo...
{"models":[{"name":"llama2:latest","model":"llama2:latest","modified_at":"2025-05-15T20:54:49.744415211Z","size":3826793677,"digest":"78e26419b4469263f75331927a00a0284ef6544c1975b826b15abdaef17bb962","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"7B","quantization_level":"Q4_0"}}]}
Descargando modelo llama2 desde Ollama...


In [3]:
#SECCIÓN 3: IMPORTACIÓN DE BIBLIOTECAS
# Estas bibliotecas proporcionan las funcionalidades esenciales para el sistema RAG

import pandas as pd
import os
import logging
import tempfile
import subprocess
import json
import requests  # Para llamadas HTTP directas a Ollama
from typing import List, Dict
import torch
import gradio as gr

# Componentes de LangChain para RAG
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Para dividir documentos
from langchain.embeddings import HuggingFaceEmbeddings  # Para crear embeddings
from langchain.vectorstores import FAISS  # Base de datos vectorial
from langchain.chains import RetrievalQA  # Framework para consultas RAG
from langchain.prompts import PromptTemplate  # Para definir prompts
from langchain_community.document_loaders import (  # Cargadores de documentos
    PyPDFLoader,  # Para PDF
    Docx2txtLoader,  # Para DOCX
    CSVLoader,  # Para CSV
    UnstructuredFileLoader  # Para texto plano y otros formatos
)
#SECCIÓN 3: IMPORTACIÓN DE BIBLIOTECAS
# Estas bibliotecas proporcionan las funcionalidades esenciales para el sistema RAG

import os
import logging
import tempfile
import subprocess
import json
import requests  # Para llamadas HTTP directas a Ollama
from typing import List, Dict
import torch
import gradio as gr

# Componentes de LangChain para RAG
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Para dividir documentos
from langchain.embeddings import HuggingFaceEmbeddings  # Para crear embeddings
from langchain.vectorstores import FAISS  # Base de datos vectorial
from langchain.chains import RetrievalQA  # Framework para consultas RAG
from langchain.prompts import PromptTemplate  # Para definir prompts
from langchain_community.document_loaders import (  # Cargadores de documentos
    PyPDFLoader,  # Para PDF
    Docx2txtLoader,  # Para DOCX
    CSVLoader,  # Para CSV
    UnstructuredFileLoader  # Para texto plano y otros formatos
)
from langchain_community.llms import Ollama  # Integración LangChain-Ollama

!pip install datasets
from datasets import load_dataset



In [16]:
from google.colab import files
files.upload()  # Aquí seleccionas tu archivo kaggle.json

import os
import shutil

# Crear la carpeta .kaggle
os.makedirs("/root/.kaggle", exist_ok=True)

# Mover kaggle.json allí
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# Cambiar permisos
os.chmod("/root/.kaggle/kaggle.json", 600)

!kaggle datasets download -d nitindatta/finance-data

!unzip -o -q finance-data.zip -d finance-data

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/nitindatta/finance-data
License(s): unknown
finance-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:
# Paso 1.1: Configurar kaggle.json (ya subido)
import os
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Paso 2: Descargar el dataset
!kaggle datasets download -d nitindatta/finance-data
!unzip -o finance-data.zip -d finance-data

# Paso 3: Cargar los datos
import pandas as pd

# Explorar archivos disponibles
print("Archivos disponibles:", os.listdir("finance-data"))

# Verificar columnas disponibles para escoger el CSV adecuado
df = pd.read_csv("finance-data/Finance_data.csv")  # Ajustado según nombre real
print("Columnas disponibles:", df.columns.tolist())

# (Si 'date' y 'headline' no existen, muestra los primeros registros)
print(df.head(3))

# Si existen columnas 'date' y 'headline', procesarlas
if 'date' in df.columns and 'headline' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m')
    df = df.dropna(subset=['date', 'headline'])
    print(df[['date', 'headline']].head())
else:
    print("⚠️ Las columnas 'date' y/o 'headline' no están presentes.")


cp: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/nitindatta/finance-data
License(s): unknown
finance-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  finance-data.zip
  inflating: finance-data/Finance_data.csv  
  inflating: finance-data/Original_data.csv  
Archivos disponibles: ['Finance_data.csv', 'Original_data.csv']
Columnas disponibles: ['gender', 'age', 'Investment_Avenues', 'Mutual_Funds', 'Equity_Market', 'Debentures', 'Government_Bonds', 'Fixed_Deposits', 'PPF', 'Gold', 'Stock_Marktet', 'Factor', 'Objective', 'Purpose', 'Duration', 'Invest_Monitor', 'Expect', 'Avenue', 'What are your savings objectives?', 'Reason_Equity', 'Reason_Mutual', 'Reason_Bonds', 'Reason_FD', 'Source']
   gender  age Investment_Avenues  Mutual_Funds  Equity_Market  Debentures  \
0  Female   34                Yes             1              2           5   
1  Female   23                Yes          

In [22]:
# SECCIÓN 4: CONFIGURACIÓN BÁSICA

# Configuración de logs para monitoreo y depuración
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Constantes para configuración del sistema
SUPPORTED_FORMATS = [".pdf", ".docx", ".doc", ".csv", ".txt"]  # Formatos soportados
EMBEDDING_MODEL = "intfloat/multilingual-e5-small"  # Modelo para codificación semántica
OLLAMA_MODEL = "llama2"  # Modelo LLM local


In [23]:
# SECCIÓN 5: CLASE PARA CARGA DE DOCUMENTOS

class DocumentLoader:
    """
    Cargador unificado de documentos que soporta múltiples formatos.
    Esta clase selecciona el cargador adecuado según la extensión del archivo.
    """

    @staticmethod
    def load_file(file_path: str) -> List:
        """
        Carga un archivo basado en su extensión y devuelve los documentos procesados.

        Args:
            file_path: Ruta al archivo a cargar

        Returns:
            Lista de documentos procesados con sus metadatos
        """
        print(f"Cargando archivo: {file_path}")
        ext = os.path.splitext(file_path)[1].lower()  # Obtener extensión del archivo

        try:
            # Seleccionar el cargador apropiado según el tipo de archivo
            if ext == '.pdf':
                loader = PyPDFLoader(file_path)  # Para archivos PDF
            elif ext in ['.docx', '.doc']:
                loader = Docx2txtLoader(file_path)  # Para documentos Word
            elif ext == '.csv':
                loader = CSVLoader(file_path)  # Para archivos CSV
            else:  # Para txt y otros formatos de texto
                loader = UnstructuredFileLoader(file_path)

            # Ejecutar la carga del documento
            documents = loader.load()

            # Enriquecer con metadatos para mejorar la recuperación y visualización
            for doc in documents:
                doc.metadata.update({
                    'title': os.path.basename(file_path),  # Nombre del archivo
                    'type': 'document',  # Tipo de contenido
                    'format': ext[1:],  # Formato sin el punto inicial
                    'language': 'auto'  # Idioma (auto-detectado)
                })

            print(f"✅ Archivo cargado exitosamente: {file_path}")
            return documents

        except Exception as e:
            print(f"❌ Error al cargar {file_path}: {str(e)}")
            raise  # Re-lanzar la excepción para manejo superior

In [24]:
# SECCIÓN 6: CLASE PRINCIPAL DEL SISTEMA RAG

class RAGSystem:
    """
    Sistema RAG completo con Ollama para consulta de documentos.

    Esta clase implementa todo el flujo de trabajo RAG:
    1. Carga y procesamiento de documentos
    2. Generación de embeddings y almacenamiento vectorial
    3. Recuperación de contexto relevante
    4. Generación de respuestas mediante LLM
    """

    def __init__(self, embedding_model: str = EMBEDDING_MODEL, ollama_model: str = OLLAMA_MODEL):
        """
        Inicializa el sistema RAG con los modelos especificados.

        Args:
            embedding_model: Modelo para generar embeddings (representaciones vectoriales)
            ollama_model: Modelo de lenguaje a utilizar con Ollama
        """
        self.embedding_model = embedding_model
        self.ollama_model = ollama_model
        self.embeddings = None  # Se inicializará posteriormente
        self.vector_store = None  # Base de datos vectorial
        self.qa_chain = None  # Cadena de pregunta-respuesta
        self.is_initialized = False  # Flag de inicialización
        self.processed_files = set()  # Conjunto para evitar procesar archivos duplicados

    def initialize_system(self):
        """
        Inicializa los componentes del sistema RAG:
        - Modelo de embeddings
        - Conexión con Ollama
        """
        try:
            print("🚀 Inicializando sistema RAG con Ollama...")

            # Inicializar el modelo de embeddings (usando CPU o GPU si está disponible)
            print("📊 Cargando modelo de embeddings...")
            self.embeddings = HuggingFaceEmbeddings(
                model_name=self.embedding_model,
                model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
                encode_kwargs={'normalize_embeddings': True}  # Normalización para mejor búsqueda
            )

            # Verificación de salud de Ollama - reintento si no responde
            try:
                response = requests.get("http://localhost:11434/api/tags")
                if response.status_code != 200:
                    print("⚠️ Advertencia: Ollama no está respondiendo correctamente. Reintentando inicialización...")
                    time.sleep(5)
                    # Reinicio de emergencia del servicio Ollama
                    subprocess.run("pkill ollama || true", shell=True)
                    subprocess.run("nohup /usr/local/bin/ollama serve > ollama_output.log 2>&1 &", shell=True)
                    time.sleep(15)  # Esperar a que reinicie
            except Exception as e:
                print(f"⚠️ Advertencia al verificar Ollama: {str(e)}")

            # Configurar Ollama como modelo de lenguaje mediante LangChain
            print("🧠 Configurando Ollama como LLM...")
            self.llm = Ollama(
                model=self.ollama_model,
                temperature=0.1,  # Temperatura baja para respuestas más deterministas
                num_predict=512  # Máximo de tokens a generar
            )

            self.is_initialized = True  # Marcar como inicializado
            print("✅ Sistema RAG inicializado correctamente")

        except Exception as e:
            print(f"❌ Error durante la inicialización: {str(e)}")
            raise

    def process_documents(self, files: List[tempfile._TemporaryFileWrapper]) -> None:
        """
        Procesa documentos cargados y actualiza la base de datos vectorial.

        Args:
            files: Lista de archivos temporales cargados por el usuario
        """
        try:
            documents = []  # Lista para almacenar todos los documentos
            new_files = []  # Seguimiento de archivos nuevos procesados

            print(f"📄 Procesando {len(files)} documento(s)...")

            # Filtrar y procesar solo archivos que no se han procesado antes
            for file in files:
                if file.name not in self.processed_files:
                    docs = DocumentLoader.load_file(file.name)  # Cargar el archivo
                    documents.extend(docs)  # Añadir documentos a la lista
                    new_files.append(file.name)  # Registrar como nuevo
                    self.processed_files.add(file.name)  # Marcar como procesado

            # Si no hay archivos nuevos, terminar
            if not new_files:
                print("ℹ️ No hay documentos nuevos para procesar")
                return

            # Verificar que se hayan cargado documentos
            if not documents:
                raise ValueError("No se pudieron cargar documentos.")

            # --------- DIVISIÓN DE DOCUMENTOS ---------
            # Dividir documentos en fragmentos más pequeños para procesamiento eficiente
            print("✂️ Dividiendo documentos en fragmentos...")
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=800,  # Tamaño objetivo de cada fragmento (en caracteres)
                chunk_overlap=200,  # Superposición entre fragmentos para mantener contexto
                separators=["\n\n", "\n", ". ", " ", ""],  # Prioridad de separación
                length_function=len  # Función para medir longitud
            )

            # Aplicar la división a todos los documentos
            chunks = text_splitter.split_documents(documents)
            print(f"🧩 Documentos divididos en {len(chunks)} fragmentos")

            # --------- VECTORIZACIÓN Y ALMACENAMIENTO ---------
            # Crear o actualizar la base de datos vectorial con los nuevos fragmentos
            print("🔍 Vectorizando fragmentos...")
            if self.vector_store is None:
                # Primera carga: crear nueva base de datos vectorial
                self.vector_store = FAISS.from_documents(chunks, self.embeddings)
            else:
                # Carga adicional: añadir a la base de datos existente
                self.vector_store.add_documents(chunks)

            # --------- CONFIGURACIÓN DE PROMPT ---------
            # Definir la plantilla de prompt para el LLM
            prompt_template = """
            Contexto: {context}

            Basándote únicamente en el contexto proporcionado, responde a la siguiente pregunta de manera clara y concisa.
            Si la información no está en el contexto, indícalo explícitamente.

            Pregunta: {question}
            """

            # Crear objeto de prompt con variables
            PROMPT = PromptTemplate(
                template=prompt_template,
                input_variables=["context", "question"]  # Variables a rellenar
            )

            # --------- CONFIGURACIÓN DE CADENA QA ---------
            # Inicializar la cadena de pregunta-respuesta con Ollama
            print("⚙️ Configurando cadena de pregunta-respuesta con Ollama...")
            self.qa_chain = RetrievalQA.from_chain_type(
                llm=self.llm,  # Modelo de lenguaje
                chain_type="stuff",  # Tipo de cadena (insertar todo el contexto de una vez)
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 6}  # Recuperar los 6 fragmentos más relevantes
                ),
                return_source_documents=True,  # Devolver documentos fuente para citas
                chain_type_kwargs={"prompt": PROMPT}  # Usar nuestro prompt personalizado
            )

            print(f"✅ Procesamiento completado: {len(documents)} documentos añadidos a la base de conocimiento")

        except Exception as e:
            print(f"❌ Error procesando documentos: {str(e)}")
            raise

    # ============================================================================
    # MÉTODO 1: GENERACIÓN MEDIANTE LANGCHAIN (más robusto)
    # ============================================================================

    def generate_response(self, question: str) -> Dict:
        """
        Genera una respuesta utilizando el framework LangChain.
        Este método es más robusto y estructurado, con mejor manejo de errores.

        Args:
            question: Pregunta del usuario

        Returns:
            Diccionario con la respuesta y fuentes utilizadas
        """
        # Verificar que el sistema esté inicializado
        if not self.is_initialized or self.vector_store is None:
            return {
                'answer': "Por favor, carga algunos documentos antes de hacer preguntas.",
                'sources': []
            }

        try:
            print(f"❓ Procesando pregunta: {question}")

            # Ejecutar la cadena QA con LangChain y Ollama
            result = self.qa_chain({"query": question})

            # Preparar la respuesta estructurada
            response = {
                'answer': result['result'],  # Respuesta generada
                'sources': []  # Lista para fuentes
            }

            # Añadir información sobre las fuentes utilizadas
            for doc in result['source_documents']:
                source = {
                    'title': doc.metadata.get('title', 'Desconocido'),
                    'content': doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content,
                    'metadata': doc.metadata
                }
                response['sources'].append(source)

            print("✅ Respuesta generada con éxito usando el método LangChain")
            return response

        except Exception as e:
            print(f"❌ Error generando respuesta con LangChain: {str(e)}")
            raise

    # ============================================================================
    # MÉTODO 2: GENERACIÓN DIRECTA CON API DE OLLAMA (más rápido)
    # ============================================================================

    def generate_with_raw_ollama(self, question: str, context: str) -> str:
        """
        Genera una respuesta usando directamente la API HTTP de Ollama.
        Este método es más rápido pero menos robusto que el método LangChain.

        Args:
            question: Pregunta del usuario
            context: Contexto recuperado de la base de conocimiento

        Returns:
            Texto de respuesta generado
        """
        try:
            # Formatear el prompt con contexto y pregunta
            formatted_prompt = f"""Contexto:
{context}

Basándote únicamente en el contexto proporcionado, responde a la siguiente pregunta de manera clara y concisa.
Si la información no está en el contexto, indícalo explícitamente.

Pregunta: {question}
"""

            # Configurar la llamada HTTP a Ollama
            headers = {"Content-Type": "application/json"}
            payload = {
                "model": self.ollama_model,
                "prompt": formatted_prompt,
                "stream": False,  # No usar streaming para simplificar
                "temperature": 0.1,  # Consistente con el otro método
                "num_predict": 512  # Número máximo de tokens
            }

            # Realizar la llamada API HTTP directa
            print("Llamando a la API de Ollama con requests...")
            response = requests.post(
                "http://localhost:11434/api/generate",
                headers=headers,
                json=payload
            )

            # Procesar la respuesta
            if response.status_code == 200:
                respuesta_json = response.json()
                respuesta = respuesta_json.get('response', 'No se obtuvo respuesta')
                return respuesta
            else:
                return f"Error en la API de Ollama: Código {response.status_code}"

        except Exception as e:
            print(f"❌ Error llamando directamente a Ollama: {str(e)}")
            # Si falla, devolver mensaje de error y sugerir usar el método estándar
            return "Error al usar Ollama directamente. Intenta desactivar 'Usar Ollama directo'."

In [25]:
# SECCIÓN 7: FUNCIÓN DE PROCESAMIENTO DE RESPUESTAS

def process_response(user_input: str, chat_history, files, use_direct_ollama=True):
    """
    Procesa la entrada del usuario y genera una respuesta utilizando el sistema RAG.
    Esta función coordina todo el proceso de consulta desde la entrada hasta la respuesta.

    Args:
        user_input: Pregunta o instrucción del usuario
        chat_history: Historial de chat actual
        files: Archivos cargados por el usuario
        use_direct_ollama: Si es True, usa la API directa de Ollama; si es False, usa LangChain

    Returns:
        Historial de chat actualizado con la nueva pregunta y respuesta
    """
    # Ignorar entradas vacías
    if not user_input.strip():
        return chat_history

    try:
        # PASO 1: Inicialización si es necesario
        if not rag_system.is_initialized:
            rag_system.initialize_system()

        # PASO 2: Procesar documentos si hay archivos nuevos
        if files:
            rag_system.process_documents(files)

        # Verificar que haya documentos procesados
        if rag_system.vector_store is None:
            answer = "Por favor, carga algunos documentos antes de hacer preguntas."
            chat_history.append((user_input, answer))
            return chat_history

        # PASO 3: Recuperar documentos relevantes para la consulta
        print("🔍 Buscando documentos relevantes...")
        documents = rag_system.vector_store.similarity_search(user_input, k=6)
        # Unir el contenido de los documentos como contexto
        context = "\n\n".join([doc.page_content for doc in documents])

        # PASO 4: Generar respuesta según el método seleccionado
        if use_direct_ollama:
            # --------- MÉTODO DIRECTO (MÁS RÁPIDO) ---------
            try:
                print("🚀 Usando método directo de Ollama...")
                answer = rag_system.generate_with_raw_ollama(user_input, context)

                # Implementación de fallback: si hay error, usar método estándar
                if answer.startswith("Error"):
                    print("⚠️ Retrocediendo al método estándar...")
                    response = rag_system.generate_response(user_input)
                    answer = response['answer']

                    # Añadir información de fuentes
                    sources = set([doc.metadata.get('title', 'Desconocido') for doc in documents[:3]])
                    if sources:
                        answer += "\n\n📚 Fuentes consultadas:\n" + "\n".join([f"• {source}" for source in sources])
            except Exception as ollama_error:
                # Manejo de error: si falla el método directo, usar el estándar
                print(f"❌ Error en método directo: {str(ollama_error)}")
                print("⚠️ Retrocediendo al método estándar...")
                response = rag_system.generate_response(user_input)
                answer = response['answer']

                # Añadir información de fuentes
                sources = set([doc.metadata.get('title', 'Desconocido') for doc in documents[:3]])
                if sources:
                    answer += "\n\n📚 Fuentes consultadas:\n" + "\n".join([f"• {source}" for source in sources])
        else:
            # --------- MÉTODO ESTÁNDAR (MÁS ROBUSTO) ---------
            print("🔄 Usando método estándar de LangChain...")
            response = rag_system.generate_response(user_input)
            answer = response['answer']

            # Añadir información de fuentes
            sources = set([doc.metadata.get('title', 'Desconocido') for doc in documents[:3]])
            if sources:
                answer += "\n\n📚 Fuentes consultadas:\n" + "\n".join([f"• {source}" for source in sources])

        # PASO 5: Actualizar el historial de chat y retornar
        chat_history.append((user_input, answer))
        return chat_history

    except Exception as e:
        # Manejo de errores generales
        error_message = f"Lo siento, ocurrió un error: {str(e)}"
        print(f"❌ Error general en process_response: {str(e)}")
        chat_history.append((user_input, error_message))
        return chat_history



In [26]:
# SECCIÓN 8: INICIALIZACIÓN DEL SISTEMA

# Crear la instancia del sistema RAG
print("🔧 Inicializando sistema RAG con Ollama...")
rag_system = RAGSystem()
print("✅ Sistema RAG creado correctamente")

🔧 Inicializando sistema RAG con Ollama...
✅ Sistema RAG creado correctamente


In [ ]:
# SECCIÓN 9: INTERFAZ GRADIO
# ============================================================================

# Crear la interfaz web con Gradio
print("🌐 Creando interfaz Gradio...")
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # Encabezado
    gr.HTML("""
        <div style="text-align: center; max-width: 800px; margin: 0 auto; padding: 20px;">
            <h1 style="color: #2d333a;">📚 RAG Assistant con Ollama</h1>
            <p style="color: #4a5568;">
                Asistente IA para análisis y consulta de documentos usando Ollama
            </p>
        </div>
    """)

    # Sección de carga de archivos y configuración
    with gr.Row():
        with gr.Column(scale=1):
            # Selector de archivos
            files = gr.Files(
                label="Carga tus documentos",
                file_types=SUPPORTED_FORMATS,
                file_count="multiple"
            )

            # Opción para seleccionar método de generación
            use_direct_ollama = gr.Checkbox(
                label="Usar Ollama directo (más rápido)",
                value=False,  # Falso por defecto para mayor estabilidad
                info="Hace llamadas directas a la API de Ollama para respuestas más rápidas."
            )

            # Información sobre formatos soportados
            gr.HTML("""
                <div style="font-size: 0.9em; color: #666; margin-top: 0.5em;">
                    Formatos soportados: PDF, DOCX, CSV, TXT
                </div>
            """)

    # Interfaz de chat
    chatbot = gr.Chatbot(
        show_label=False,
        container=True,
        height=500,
        bubble_full_width=False,
        show_copy_button=True,
        scale=2
    )

    # Área de entrada de texto y botón de limpieza
    with gr.Row():
        message = gr.Textbox(
            placeholder="💭 Pregunta cualquier cosa sobre tus documentos...",
            show_label=False,
            container=False,
            scale=8,
            autofocus=True
        )
        clear = gr.Button("🗑️ Limpiar", size="sm", scale=1)

    # Sección de instrucciones
    gr.HTML("""
        <div style="background-color: #f8f9fa; padding: 15px; border-radius: 10px; margin: 20px 0;">
            <h3 style="color: #2d333a; margin-bottom: 10px;">🔍 Cómo usar:</h3>
            <ol style="color: #666; margin-left: 20px;">
                <li>Carga uno o más documentos (PDF, DOCX, CSV, o TXT)</li>
                <li>Espera a que los documentos sean procesados</li>
                <li>Haz preguntas sobre el contenido de tus documentos</li>
                <li>Activa "Usar Ollama directo" para respuestas más rápidas (desactívalo si hay errores)</li>
            </ol>
            <p style="color: #666; font-style: italic; margin-top: 10px;">
                Nota: La primera respuesta puede tardar un poco. Desde la segunda respuesta es más rápido.
            </p>
        </div>
    """)

    # Pie de página con información técnica y créditos
    gr.HTML("""
        <div style="text-align: center; max-width: 800px; margin: 20px auto; padding: 20px;
                    background-color: #f8f9fa; border-radius: 10px;">
            <div style="margin-bottom: 15px;">
                <h3 style="color: #2d333a;">⚡ Sobre este asistente</h3>
                <p style="color: #666; font-size: 14px;">
                    Esta aplicación utiliza tecnología RAG (Retrieval Augmented Generation) combinando:
                </p>
                <ul style="list-style: none; color: #666; font-size: 14px;">
                    <li>🔹 Motor LLM: Ollama con Llama2</li>
                    <li>🔹 Embeddings: multilingual-e5-small</li>
                    <li>🔹 Base de datos vectorial: FAISS</li>
                </ul>
            </div>
            <div style="border-top: 1px solid #ddd; padding-top: 15px;">
                <p style="color: #666; font-size: 14px;">
                    Creado para el curso de Inteligencia Artificial Aplicada - Universidad de los Andes<br>
                    Por <a href="https://www.linkedin.com/in/camilovegabarbosa/"
                    target="_blank" style="color: #2196F3; text-decoration: none;">Camilo Vega</a>,
                    Profesor de IA 🤖
                </p>
            </div>
        </div>
    """)

    # --------- FUNCIONES DE CONTROL DE LA INTERFAZ ---------
    # Función para limpiar el contexto y reiniciar
    def clear_context():
        # Eliminar la base de conocimiento y reiniciar el registro de archivos
        rag_system.vector_store = None
        rag_system.processed_files.clear()
        return None

    # Conectar eventos de la interfaz con funciones
    message.submit(process_response, [message, chatbot, files, use_direct_ollama], [chatbot])
    clear.click(clear_context, None, chatbot)

# Lanzar la interfaz web
print("🚀 Lanzando interfaz Gradio...")
demo.launch(share=True, debug=True)

🌐 Creando interfaz Gradio...
🚀 Lanzando interfaz Gradio...


<ipython-input-27-43c02599d002>:42: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
<ipython-input-27-43c02599d002>:42: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://32964197637239b696.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
